In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Embedding,LSTM,Dense


In [21]:
data=pd.read_csv("medical_data.csv")
print(data.head)

<bound method NDFrame.head of                                        Patient_Problem  \
0    Constant fatigue and muscle weakness, struggli...   
1    Frequent severe migraines, sensitivity to ligh...   
2    Sudden weight gain and feeling cold, especiall...   
3    High fever, sore throat, and swollen lymph nod...   
4    Excessive thirst and frequent urination, dry m...   
..                                                 ...   
402  Noticeable thinning of the hair on the top of ...   
403  Greenish discharge from the eyes, accompanied ...   
404  Experiencing confusion, difficulty speaking, a...   
405  Constant feeling of fullness in the ears and h...   
406  Blood in stool, along with a change in bowel m...   

                         Disease  \
0       Chronic Fatigue Syndrome   
1             Migraine with Aura   
2                 Hypothyroidism   
3                  Mononucleosis   
4              Diabetes Mellitus   
..                           ...   
402        Androgenet

In [23]:
tokenizer=Tokenizer(num_words=5000,oov_token="<OOV>")
tokenizer.fit_on_texts(data["Patient_Problem"])
sequences=tokenizer.texts_to_sequences(data["Patient_Problem"])

In [25]:
max_length=max(len(x) for x in sequences)
padded_sequences=pad_sequences(sequences,maxlen=max_length,padding="post")

In [27]:
label_encoder_disease=LabelEncoder()
label_encoder_prescription=LabelEncoder()
disease_labels=label_encoder_disease.fit_transform(data["Disease"])
prescription_labels=label_encoder_prescription.fit_transform(data["Prescription"])
disease_labels_categorical=to_categorical(disease_labels)
prescription_labels_categorical=to_categorical(prescription_labels)

In [29]:
y=np.hstack((disease_labels_categorical,prescription_labels_categorical))


In [33]:
input_layer=Input(shape=(max_length,))
embedding=Embedding(input_dim=5000,output_dim=64)(input_layer)
lstm_layer=LSTM(64)(embedding)
disease_output=Dense(len(label_encoder_disease.classes_),activation="softmax",name="disease_output")(lstm_layer)
prescription_output=Dense(len(label_encoder_prescription.classes_),activation="softmax",name="prescription_output")(lstm_layer)


In [37]:
model=Model(inputs=input_layer,outputs=[disease_output,prescription_output])
model.compile(
    loss={"disease_output":"categorical_crossentropy",
          "prescription_output":"categorical_crossentropy"},
    optimizer="adam",
    metrics={"disease_output":["accuracy"],"prescription_output":["accuracy"]}
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 17)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 17, 64)       320000      ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    (None, 64)           33024       ['embedding[0][0]']              
                                                                                                  
 disease_output (Dense)         (None, 178)          11570       ['lstm[0][0]']                   
                                                                                              

In [39]:
model.fit(padded_sequences,{"disease_output":disease_labels_categorical,"prescription_output":
                            prescription_labels_categorical},epochs=100,batch_size=32)

Epoch 1/100
13/13 [==============================] - 3s 16ms/step - loss: 11.1469 - disease_output_loss: 5.1812 - prescription_output_loss: 5.9656 - disease_output_accuracy: 0.0074 - prescription_output_accuracy: 0.0000e+00
Epoch 2/100
13/13 [==============================] - 0s 15ms/step - loss: 11.1301 - disease_output_loss: 5.1680 - prescription_output_loss: 5.9621 - disease_output_accuracy: 0.0098 - prescription_output_accuracy: 0.0025
Epoch 3/100
13/13 [==============================] - 0s 16ms/step - loss: 11.1089 - disease_output_loss: 5.1455 - prescription_output_loss: 5.9633 - disease_output_accuracy: 0.0270 - prescription_output_accuracy: 0.0000e+00
Epoch 4/100
13/13 [==============================] - 0s 18ms/step - loss: 11.0122 - disease_output_loss: 5.0327 - prescription_output_loss: 5.9794 - disease_output_accuracy: 0.0344 - prescription_output_accuracy: 0.0000e+00
Epoch 5/100
13/13 [==============================] - 0s 16ms/step - loss: 10.9057 - disease_output_loss: 4.9

In [45]:
def make_prediction(patient_problem):
    sequence=tokenizer.texts_to_sequences([patient_problem])
    padded_sequence=pad_sequences(sequence,maxlen=max_length,padding="post")
    prediction=model.predict(padded_sequence)
    disease_index=np.argmax(prediction[0],axis=1)[0]
    prescription_index=np.argmax(prediction[1],axis=1)[0]
    disease_predicted=label_encoder_disease.inverse_transform([disease_index])[0]
    prescription_predicted=label_encoder_prescription.inverse_transform([prescription_index])[0]
    print(f"Predicted Disease:{disease_predicted}")
    print(f"Predicted Prescription:{prescription_predicted}")
patient_input="I've experienced a loss of appetite  and don't enjoy food anymore."
make_prediction(patient_input)

1/1 [==============================] - 0s 41ms/step
Predicted Disease:Depression
Predicted Prescription:Antidepressants; eating nutrient-rich foods.
